<a href="https://colab.research.google.com/github/analeticiagarcez/Breast_cancer_segmentation_and_classification/blob/features/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown https://cloud.andrelab.icmc.usp.br/s/xX2SbG4cLiNCPMA/download/archive.zip
!unzip archive.zip

# Tratamento das saídas

EXPLICAÇÃO

In [43]:
import os
import re
import cv2
from google.colab.patches import cv2_imshow

path_dataset = '/content/Dataset_BUSI_with_GT/'
classes = os.listdir(path_dataset)

# Itera sobre as classes
for c in classes:
  # Monta o path para a pasta da classe
  path_class = os.path.join(path_dataset, c)
  # Pega o nome dos arquivos
  nome_imagens = os.listdir(path_class)
  for nome_imagem in nome_imagens:
    if 'mask_' in nome_imagem:
      # Escrevo o nome da nova imagem
      nome_imagem_original = re.sub('_[0-9]+', '', nome_imagem)
      # Cria o path das máscaras
      path_imagem_original =  os.path.join(path_class, nome_imagem_original)
      path_imagem_atual =  os.path.join(path_class, nome_imagem)
      # Abre as imagens
      img_original = cv2.imread(path_imagem_original)
      img_atual = cv2.imread(path_imagem_atual)
      img_somada = cv2.add(img_original, img_atual)
      # Sobrescreve o arquivo com as imagens fundidas
      cv2.imwrite(path_imagem_original, img_somada)

# Leitura das imagens

In [79]:
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class DadosImagens(Dataset):
  def __init__(self, path, image_size):
    self.files_path = path
    self.imagens = []
    self.masks = []
    self.image_class = []
    self.__transform(image_size)
    #Lista as classes dos dados
    classes = os.listdir(self.files_path)
    # Itera sobre as classes
    for c in classes:
      # Monta o path para a pasta da classe
      path_class = os.path.join(path_dataset, c)
      # Pega o nome dos arquivos
      nome_imagens = os.listdir(path_class)
      for nome_imagem in nome_imagens:
        if 'mask' in nome_imagem:
          continue
        # Monta o caminho para a imagem de entrada e a mácara
        path_imagem = os.path.join(path_class, nome_imagem)
        path_mask = path_imagem.replace('.', '_mask.')
        # Faz a leitura da máscara
        imagem = Image.open(path_imagem)
        mask =  Image.open(path_mask)
        # Salva as imagens na lista
        self.imagens.append(imagem)
        self.masks.append(mask)
        self.image_class.append(c)

  def __transform(self, image_size):
    # Inicializa o transormador dos dados
    self.transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

  def __len__(self):
    return len(self.imagens)

  def __getitem__(self, idx):
    # Pega as imagens no índice
    imagem = self.imagens[idx]
    mask = self.masks[idx]
    classe = self.image_class[idx]
    # Transforma as imagens
    imagem = self.transform(imagem)
    mask = self.transform(mask)
    # Retorna os dados das imagens
    return {'img': imagem, 'mask': mask, 'label': classe}

dados = DadosImagens(path_dataset, (256,256))

In [ ]:
import torchvision.models

model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=1, out_channels=1, init_features=32, pretrained=True)

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


RuntimeError: ignored